In [10]:
%pip install spacy
import spacy
!python3.11 -m spacy download en_core_web_sm

%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
import json
import os
import spacy
from spacy.tokens import Doc
from collections import defaultdict
from typing import List, Tuple

import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/shaojh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/shaojh/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [13]:
#test

with open('translations/wolof.json', 'r') as fp:
    wolof = json.load(fp)
print(wolof['passages']['1']['official'])
print(wolof['passages']['1']['roundtrip_chatgpt'])
print(wolof['passages']['1']['roundtrip_google'])

Ca njàlbéen ga Yàlla sàkk na asamaan ak suuf. Suuf nag daa maase woon te wéet, lëndëm muur ndox mu xóot mi, lale ca kaw, Noowug Yàlla di wër, tiim ndox mi.Ba loolu amee Yàlla ne: «Na leer nekk,» leer daldi am. Yàlla gis ne leer gi lu baax la, mu xàjjale leer ak lëndëm. Yàlla tudde leer gi bëccëg, lëndëm gi guddi. Ngoon jot, suba dugg, muy bés ba jëkk.
In the beginning, God created the heavens and the earth. The earth had no form and was empty; and darkness was over the surface of the deep. And the Spirit of God was hovering over the waters. And God said, 'Let there be light.' And there was light. And God saw that the light was good; and He separated the light from the darkness. God called the light 'Day,' and He called the darkness 'Night.' And there was evening and there was morning—the first day.
In the beginning God created the heavens and the earth. And the earth was without form, and void; Darkness covered the deep. And the Spirit of God moved upon the face of the waters. And God 

In [18]:
nlp = spacy.load("en_core_web_sm")

# parse text into syntax tree
def extract_dependencies(text: str) -> List[Tuple[str, str, str]]:
    doc = nlp(text)
    deps = []
    for sent in doc.sents:
        for token in sent:
            deps.append((token.text, token.dep_, token.head.text))
    return deps

# compare two dependency trees by counting matching dependency triples
def compare_trees(deps1, deps2):
    matches = 0
    for word1, dep1, head1 in deps1:
        for word2, dep2, head2 in deps2:
            if dep1 == dep2 and are_synonyms(word1, word2) and are_synonyms(head1, head2):
                matches += 1
                break
    return matches / max(len(deps1), len(deps2)) if deps1 and deps2 else 0

# detects if two words are synonyms
def are_synonyms(word1, word2):
    similarity_threshold = 0.75

    if word1.lower() == word2.lower():
        return True
    
    syns1 = wn.synsets(word1)
    syns2 = wn.synsets(word2)
    
    for s1 in syns1:
        for s2 in syns2:
            if s1.wup_similarity(s2) and s1.wup_similarity(s2) > similarity_threshold:
                return True
    return False

In [19]:
language_strings = ["malay", "nahuatl", "quechua",  "wolof"] #Tibetan removed due to formatting errors
language_jsons = {}
for language_string in language_strings:
    language_json_string = "translations/" + language_string + ".json"

    with open(language_json_string, 'r', encoding='utf-8') as fp:
        language_json = json.load(fp)
        language_jsons[language_string] = language_json

        
# print(language_jsons)

for language_string, language_json in language_jsons.items():

    print("\n" + language_string + "\n")

    for passage_number in language_json['passages']:

        passage = language_json['passages'][passage_number]

        print(passage)

        english = passage['english']
        roundtrip_chatgpt = passage['roundtrip_chatgpt']
        roundtrip_google = passage['roundtrip_google']
        
        print(english)
        print(roundtrip_chatgpt)
        print(roundtrip_google)

        # extract dependency trees
        english_deps = extract_dependencies(english)
        chatgpt_deps = extract_dependencies(roundtrip_chatgpt)
        google_deps = extract_dependencies(roundtrip_google)

        # compare trees
        score_chatgpt = compare_trees(english_deps, chatgpt_deps)
        score_google = compare_trees(english_deps, google_deps)

        # print the comparison
        print("Similarity with ChatGPT roundtrip:", score_chatgpt)
        print("Similarity with Google roundtrip:", score_google)

        # print out the trees
        print("\nChatGPT dependencies:")
        for dep in chatgpt_deps[:10]:  # limit output
            print(dep)

        print("\nGoogle dependencies:")
        for dep in google_deps[:10]:
            print(dep)



malay

{'english': 'In the beginning God created the heaven and the earth. And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters. And God said, Let there be light: and there was light. And God saw the light, that it was good: and God divided the light from the darkness. And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.', 'official': 'Waktu pertama skali, Allah bikin langit deng bumi ni smua. Waktu itu, bumi ni masi trada bentuk, trada apapa ada air saja, baru smua glap. Roh Allah ada mlayang-mlayang di atas air situ. Trus begini Allah bilang, “Sa mau ada trang skarang!” Langsung trang de jadi. Allah liat kalo trang ni de bagus. Baru trang deng glap ni, Allah kas pisa dorang. Allah kas nama trang tu ʻsiangʼ baru glap tu ʻmalam.ʼ Suda, malam su lewat trus pagi muncul. Begitu suda, hari pertama de jadi.', 'chatgpt': 'Pada mulanya Tuhan 